In [1]:
import pickle
experiment_path = 'results/SBM/SBM_MC_tanh_nnodes50_p10.5_delta_p0.5_min_weight0_bin_directedTrue_wei_directedFalse/'
network_data_path = experiment_path + 'network_data/'
nets = pickle.load(open(network_data_path + 'nets.pickle', 'rb'))

In [2]:
import os
import json
import numpy as np

#Get pruning_ratio in config.txt
with open(os.path.join(experiment_path, 'config.txt'), 'r') as f:
    config = json.load(f)
    pruning_ratio = config['pruning_ratio']

pruned_nets = {}
for net_type, net_arr in nets.items():
    pruned_nets['level' + net_type] = []
    for net in net_arr:
        new_net = net.copy()
        idx = np.where(new_net != 0)
        nedges = len(idx[0])
        nprune = int(pruning_ratio*nedges)
        idx_prune = np.random.choice(range(nedges),
                                     nprune, replace=False)
        new_net[idx[0][idx_prune], idx[1][idx_prune]] = 0
        bin_net = np.where(new_net != 0, 1., 0.)
        pruned_nets['level' + net_type].append(bin_net)

In [ ]:
#Save pruned nets for both Python and Matlab
with open(os.path.join(network_data_path, 'pruned_nets.pickle'), 'wb') as f:
    pickle.dump(pruned_nets, f)
    
import scipy.io
scipy.io.savemat(os.path.join(network_data_path, 'pruned_nets.mat'),
                 pruned_nets)

In [4]:
#Compute the clustering coefficient
import bct
cc_dict = {}
for net_type, net_arr in pruned_nets.items():
    if net_type != 'levelMS':
        cc_dict[net_type] = []
        for net in net_arr:
            cc = bct.clustering_coef_bd(net)
            #network average
            cc_dict[net_type].append(np.mean(cc))

In [5]:
#Transform the dictionary to a dataframe
import pandas as pd
cc_df = pd.DataFrame(cc_dict).melt(var_name='level', value_name='cc')
pickle.dump(cc_df, open(os.path.join(network_data_path, 'cc_df.pickle'), 'wb'))

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

from score_plotting_utils import create_fig_path

sns.set_style("ticks")
sns.set(context=None, style=None, palette=None, font_scale=5, color_codes=None)
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({'font.size': 15})
plt.rcParams['legend.fontsize'] = 15

PuBu = cm.get_cmap('PuBu')
GnBu = cm.get_cmap('GnBu')
PuRd = cm.get_cmap('PuRd')
palette = [PuBu(0.6), GnBu(0.6), PuRd(0.6)]

In [7]:
fig_path = create_fig_path(experiment_path)

ax = sns.kdeplot(data=cc_df, x='cc', hue='level', 
                 palette=palette, fill=True, cut=0, alpha=0.8)
ax.set_box_aspect(1)

ax.set_xlabel('Clustering coefficient')
ax.set_ylabel('Density')

fig = ax.get_figure()
fig.savefig(os.path.join(fig_path, 'cc.png'), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(fig_path, 'cc.svg'), dpi=300, bbox_inches='tight')
plt.close(fig)

In [8]:
from score_plotting_utils import mannwhitneyu_print
for xlevel, ylevel in zip(['level1', 'level1', 'level2'],
                          ['level2', 'level3', 'level3']):
    x = cc_df[cc_df['level'] == xlevel].cc
    y = cc_df[cc_df['level'] == ylevel].cc
    x_stats, y_stats, MWU_stats, cles_stats = mannwhitneyu_print(x, y, xlevel, ylevel)
    print(f'{x_stats}\n{y_stats}\n{MWU_stats}\n{cles_stats}\n\n')

level1 median=0.12018766742722048, IQR=0.0015600229642583063
level2 median=0.1325289329607644, IQR=0.0013791112912671377
Mann-Whitney U rank test level1-level2: u=0.0, p=2.562143669163401e-34
cles = 0.0


level1 median=0.12018766742722048, IQR=0.0015600229642583063
level3 median=0.13617502882199964, IQR=0.0016747022655143884
Mann-Whitney U rank test level1-level3: u=0.0, p=2.562143669163401e-34
cles = 0.0


level2 median=0.1325289329607644, IQR=0.0013791112912671377
level3 median=0.13617502882199964, IQR=0.0016747022655143884
Mann-Whitney U rank test level2-level3: u=122.0, p=9.579572699541495e-33
cles = 0.0122




In [9]:
#MOTIFS
pruned_nets_motifs = scipy.io.loadmat(os.path.join(network_data_path, 'pruned_nets_motifs.mat'))
func_motifs = pruned_nets_motifs['F_func'].copy()

func_motif_dict = {'F': [F for net in range(len(func_motifs)) for F in func_motifs[net, :]],
                   'motif': [motif + 1 for net in range(len(func_motifs)) for motif in range(func_motifs.shape[1])],
                   'network': [net for net in range(len(func_motifs)) for motif in range(func_motifs.shape[1])],
                   'level': [level[0][0] for level in pruned_nets_motifs['level_names'] for motif in range(func_motifs.shape[1])]}

import pandas as pd
func_motif_df = pd.DataFrame(func_motif_dict)

In [10]:
#Normalize F by its sum in each network
freq_func_motif_df = func_motif_df.copy(deep=True)
freq_func_motif_df['F'] = freq_func_motif_df.groupby('network')['F'].transform(lambda x: x / x.sum())
pickle.dump(freq_func_motif_df, open(os.path.join(network_data_path, 'freq_func_motif_df.pickle'), 'wb'))

In [11]:
fig, ax = plt.subplots(figsize=(5, 5))
sns.barplot(data=freq_func_motif_df[(freq_func_motif_df.level != 'MS') & (freq_func_motif_df.motif <= 3)], 
            x='motif', y='F', hue='level', palette=palette, seed=0, ax=ax)

ax.set_xlabel('Functional Motif')
ax.set_ylabel('Frequency')

fig.savefig(os.path.join(fig_path, 'freq_func_motifs123.png'), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(fig_path, 'freq_func_motifs123.svg'), dpi=300, bbox_inches='tight')
plt.close(fig)

In [12]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(data=freq_func_motif_df[(freq_func_motif_df.level != 'MS') & (freq_func_motif_df.motif > 3)], 
            x='motif', y='F', hue='level', palette=palette, seed=0, ax=ax)

ax.set_xlabel('Functional Motif')
ax.set_ylabel('Frequency')

fig.savefig(os.path.join(fig_path, 'freq_func_motifs.png'), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(fig_path, 'freq_func_motifs.svg'), dpi=300, bbox_inches='tight')
plt.close(fig)

In [13]:
for motif in range(1, 14):
    print(f'Motif {motif}')
    for xlevel, ylevel in zip(['level1', 'level1', 'level2'],
                              ['level2', 'level3', 'level3']):
        x = freq_func_motif_df[(freq_func_motif_df.level == xlevel) & (freq_func_motif_df.motif == motif)]['F']
        y = freq_func_motif_df[(freq_func_motif_df.level == ylevel) & (freq_func_motif_df.motif == motif)]['F']
        x_stats, y_stats, MWU_stats, cles_stats = mannwhitneyu_print(x, y, xlevel, ylevel)
        print(f'{x_stats}\n{y_stats}\n{MWU_stats}\n{cles_stats}\n\n')

Motif 1
level1 median=0.1904707072848063, IQR=0.0012591296332766233
level2 median=0.18572740628594903, IQR=0.0012640748213267605
Mann-Whitney U rank test level1-level2: u=9987.0, p=3.7845631983265776e-34
cles = 0.9987


level1 median=0.1904707072848063, IQR=0.0012591296332766233
level3 median=0.18452222787594713, IQR=0.001319355944675693
Mann-Whitney U rank test level1-level3: u=10000.0, p=2.562143669163401e-34
cles = 1.0


level2 median=0.18572740628594903, IQR=0.0012640748213267605
level3 median=0.18452222787594713, IQR=0.001319355944675693
Mann-Whitney U rank test level2-level3: u=8040.0, p=1.1135688416166574e-13
cles = 0.804


Motif 2
level1 median=0.35302662048291344, IQR=0.0023810507219203525
level2 median=0.3417392399482889, IQR=0.0024441490494975793
Mann-Whitney U rank test level1-level2: u=10000.0, p=2.562143669163401e-34
cles = 1.0


level1 median=0.35302662048291344, IQR=0.0023810507219203525
level3 median=0.3385520875502645, IQR=0.002249290509666546
Mann-Whitney U rank test

In [14]:
#CYCLES
ncycles_level1 = pickle.load(open(os.path.join(network_data_path, 'ncycles_level1.pickle'), 'rb'))
ncycles_level2 = pickle.load(open(os.path.join(network_data_path, 'ncycles_level2.pickle'), 'rb'))
ncycles_level3 = pickle.load(open(os.path.join(network_data_path, 'ncycles_level3.pickle'), 'rb'))

In [15]:
data = {'level1': ncycles_level1, 'level2': ncycles_level2, 'level3': ncycles_level3}
ncycles_dict = {'cycle_len': [], 'n_cycles': [], 'level': []}
for level, data_dict_list in data.items():
    for data_dict in data_dict_list:
        for cycle_len, n_cycles in data_dict.items():
            if cycle_len < 3:
                continue
            ncycles_dict['cycle_len'].append(cycle_len)
            ncycles_dict['n_cycles'].append(n_cycles)
            ncycles_dict['level'].append(level)

ncycles_df = pd.DataFrame(ncycles_dict)
pickle.dump(ncycles_df, open(os.path.join(network_data_path, 'ncycles_df.pickle'), 'wb'))

In [16]:
for cycle_len in range(3, 6):
    print(f'Cycle length {cycle_len}')
    
    #plotting
    ax = sns.kdeplot(data=ncycles_df[ncycles_df.cycle_len == cycle_len], 
                     x='n_cycles', hue='level', palette=palette, 
                     fill=True, cut=0, alpha=0.8)
    ax.set_box_aspect(1)

    ax.set_xlabel('Number of cycles')
    ax.set_ylabel('Density')

    fig = ax.get_figure()
    fig.savefig(os.path.join(fig_path, f'ncycles{cycle_len}.png'), dpi=300, bbox_inches='tight')
    fig.savefig(os.path.join(fig_path, f'ncycles{cycle_len}.svg'), dpi=300, bbox_inches='tight')
    plt.close(fig)

    #stats
    for xlevel, ylevel in zip(['level1', 'level1', 'level2'],
                              ['level2', 'level3', 'level3']):
        x = ncycles_df[(ncycles_df.level == xlevel) & (ncycles_df.cycle_len == cycle_len)].n_cycles
        y = ncycles_df[(ncycles_df.level == ylevel) & (ncycles_df.cycle_len == cycle_len)].n_cycles
        x_stats, y_stats, MWU_stats, cles_stats = mannwhitneyu_print(x, y, xlevel, ylevel)
        print(f'{x_stats}\n{y_stats}\n{MWU_stats}\n{cles_stats}\n\n')

Cycle length 3
level1 median=72876.5, IQR=1568.25
level2 median=78246.5, IQR=1605.75
Mann-Whitney U rank test level1-level2: u=3.0, p=2.8032637239356287e-34
cles = 0.0003


level1 median=72876.5, IQR=1568.25
level3 median=80317.0, IQR=2041.25
Mann-Whitney U rank test level1-level3: u=0.0, p=2.5618550313305253e-34
cles = 0.0


level2 median=78246.5, IQR=1605.75
level3 median=80317.0, IQR=2041.25
Mann-Whitney U rank test level2-level3: u=1280.0, p=1.0069663759725663e-19
cles = 0.128


Cycle length 4
level1 median=3577346.0, IQR=93508.25
level2 median=3693850.5, IQR=114276.5
Mann-Whitney U rank test level1-level2: u=1426.0, p=2.5142775420431724e-18
cles = 0.1426


level1 median=3577346.0, IQR=93508.25
level3 median=3782905.0, IQR=116516.5
Mann-Whitney U rank test level1-level3: u=241.0, p=3.009446242346512e-31
cles = 0.0241


level2 median=3693850.5, IQR=114276.5
level3 median=3782905.0, IQR=116516.5
Mann-Whitney U rank test level2-level3: u=2034.0, p=4.296729002134073e-13
cles = 0.2034

